86 Variable about on product usage
and socio-demographic data

5822 in training set
4000 in test set

__Predict who will buy caravan insurance policy__

Computer the Ordinary Least Square estimates and compare them with those obtained from 
1. Forwards Selection
2. Backwards Selection
3. Lasso Regression
4. Ridge Regression


In [26]:
import csv
import time
import pandas as pd
from sklearn.feature_selection import f_regression
from sklearn.model_selection import cross_val_score, KFold, cross_validate
from sklearn.linear_model import Ridge, Lasso, SGDRegressor
from sklearn.linear_model import LinearRegression 


from mlxtend.feature_selection import SequentialFeatureSelector as SFS # includes forward and backwawrds selection
#https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#example-2-toggling-between-sfs-sbs-sffs-and-sbfs


In [2]:

#Read Data Complete
training_data = pd.read_csv('ticdata2000.txt', sep="\t", header=None)
test_data = pd.read_csv('ticeval2000.txt' , sep='\t', header = None)
target_data = pd.read_csv('tictgts2000.txt' , sep='\t', header = None)

In [3]:
len(training_data), len(test_data), len(target_data)

training_data_column_count = training_data.shape[1]
test_data_column_count = test_data.shape[1]
target_data_column_count = target_data.shape[1]

In [4]:
training_data.shape, test_data.shape, target_data.shape

((5822, 86), (4000, 85), (4000, 1))

Name Cell Columns

In [5]:
train_data_column=[]
test_data_column = []
target_data_column = []
for i in range(1, training_data_column_count):
    feature_name = 'Feature_' + str(i)
    train_data_column.append(feature_name)
    test_data_column.append(feature_name)
#print(train_data_column)
train_data_column.append("target_caravan")
target_data_column.append("target_caravan")

training_data.columns = train_data_column
test_data.columns = test_data_column
target_data.columns = target_data_column

training_data_target = training_data.iloc[:,-1]
training_data_feature = training_data.iloc[:, 0:training_data_column_count-1]

Data Process Complete

In [18]:
# OLS
from sklearn.linear_model import LinearRegression as lm
lm = LinearRegression()

model=lm.fit(training_data_feature, training_data_target) 
results = cross_val_score(model, test_data, target_data, cv = 10)
max_value = 0 
for i in results:
    if i > max_value:
        max_value = i
print("Test Accuracy for OLS is " ,max_value)


Test Accuracy for OLS is  0.0853462345424455


In [27]:
# Sequential Forward Selection

lm = LinearRegression()

start = time.time() 

sfs = SFS(lm, 
          k_features=3, 
          forward=True, 
          floating=False, 
          cv=4,
          n_jobs=-1)

sfs = sfs.fit(training_data_feature, training_data_target)

end = time.time()
print(' runtime = %f'%( end-start))
print('\nSequential Forward Selection (k=3):')
print(sfs.k_feature_idx_)
print('CV Score:')
print((sfs.k_score_*100))


 runtime = 1.342415

Sequential Forward Selection (k=3):
(42, 46, 81)
CV Score:
3.956031906280372


__Forward Selection Output__ <br>
Sequential Forward Selection (k=3): <br>
(42, 46, 81)<br>
CV Score:<br>
3.956031906280372<br>

Sequential Forward Selection (k=3):<br>
(9, 15, 20, 40, 42, 43, 45, 46, 58, 81)<br>
CV Score:<br>
5.354115734215675<br>


In [28]:

# Sequential Backward Selection

lm = LinearRegression()

start = time.time() 
sbs = SFS(lm, 
          k_features=3, 
          forward=False, 
          floating=False, 
          cv=4,
          n_jobs=-1) 

sbs = sbs.fit(training_data_feature, training_data_target)

end = time.time()
print(' runtime = %f'%( end-start))
print('\nSequential Backward Selection (k=3):')
print(sbs.k_feature_idx_)
print('CV Score:')
print((sbs.k_score_*100))


 runtime = 173.865419

Sequential Backward Selection (k=3):
(17, 46, 81)
CV Score:
3.9023624397377303


__Backward Selection Output__ <br>
Sequential Backward Selection (k=3):<br>
(17, 46, 81)<br>
CV Score:<br>
3.9023624397377303<br>


In [15]:
# Lasso 
for k in [0, 0.01, 0.1]:
    lasso = Lasso(alpha=k, max_iter=100000,fit_intercept=False, tol=0.000001 )
    results = cross_val_score(lasso, training_data_feature, training_data_target, cv = 10)
#     print(results)

print("Accuracy, " , results.mean() * 100)

C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 135.89113403964728, tolerance: 0.00031299999999999996
  positive)
C:\Users\Matthew\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
C:

Accuracy,  1.3054810076146928


In [16]:
# Ridge 
# kfold cross validation score 
for k in [0, 0.01, 0.1]:
    ridge = Ridge(alpha=k)
    results = cross_val_score(ridge, training_data_feature, training_data_target, cv = 10) 
print("Accuracy, " , results.mean() * 100)

Accuracy,  3.017164024976703
